In [1]:
!pip install Boruta

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


$$\text{Dữ liệu Gốc} \rightarrow \text{EDA} \rightarrow \text{Xử lý Giá trị Thiếu/Encoding} 

\rightarrow \text{TRAIN-TEST SPLIT} \rightarrow \text{Scaling/PCA} \rightarrow \text{Training}

 \rightarrow \text{Evaluation}$$

In [ ]:

from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

# instantiate random forest
forest = RandomForestRegressor(n_jobs = -1, max_depth = 5)

# fit boruta
boruta_selector = BorutaPy(forest, n_estimators = 'auto', random_state = 0)
boruta_selector.fit(np.array(X_trn), np.array(y_trn))

# store results
boruta_ranking = boruta_selector.ranking_
selected_features = np.array(feature_names)[boruta_ranking <= 2]


# Unsupervise

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ==============================================================================
# 1. TẠO DỮ LIỆU GIẢ LẬP VÀ XÁC ĐỊNH CỘT (Thay thế bằng data thực của bạn)
# ==============================================================================
np.random.seed(42)
data_size = 500
df = pd.DataFrame({
    # 5 Cột số (Num): A và D tương quan cao, E phương sai thấp
    'Num_A': np.random.rand(data_size) * 10,
    'Num_B': np.random.normal(50, 10, data_size),
    'Num_C': (np.random.rand(data_size) * 0.9) + (np.random.rand(data_size) * 0.8), 
    'Num_D': (np.random.rand(data_size) * 0.1) + (df['Num_A'] * 0.95), # Tương quan rất cao với Num_A
    'Num_E': np.random.rand(data_size) * 0.001 + 100, # Phương sai thấp

    # 4 Cột phân loại (Cat): H có tần suất thống trị (>95%)
    'Cat_F': np.random.choice(['Red', 'Green', 'Blue'], data_size, p=[0.7, 0.2, 0.1]),
    'Cat_G': np.random.choice(['A', 'B', 'C', 'D'], data_size, p=[0.4, 0.3, 0.2, 0.1]),
    'Cat_H': ['High'] * 490 + ['Low'] * 10, 
    'Cat_I': np.random.choice(['X', 'Y', 'Z'], data_size)
})
# Thêm 40 cột giả lập khác để mô phỏng 49 cột
for i in range(1, 41):
    df[f'Other_Col_{i}'] = np.random.rand(data_size)

# Loại bỏ các cột không phải số/chuỗi nếu có
df = df.select_dtypes(include=['number', 'object'])
print(f"Tổng số cột ban đầu: {df.shape[1]}")

# Phân loại cột ban đầu
num_cols = df.select_dtypes(include=np.number).columns.tolist()
cat_cols = df.select_dtypes(include='object').columns.tolist()
print("-" * 50)


# ==============================================================================
# 2. EDA VÀ LỰA CHỌN BIẾN (FEATURE SELECTION) TRÊN TOÀN BỘ DỮ LIỆU GỐC
# ==============================================================================

# Khởi tạo tập hợp các cột cần loại bỏ
cols_to_drop = set()

# --- 2.1. Loại bỏ Missing Values (>50%) ---
missing_percentage = (df.isnull().sum() / len(df)) * 100
cols_to_drop_missing = missing_percentage[missing_percentage > 50].index.tolist()
cols_to_drop.update(cols_to_drop_missing)

if cols_to_drop_missing:
    print(f"🚫 Loại bỏ {len(cols_to_drop_missing)} cột có >50% Missing Values.")
else:
    print("✅ Không có cột nào bị loại bỏ do Missing Values quá cao.")

# Cập nhật danh sách cột sau khi loại bỏ Missing
num_cols = [col for col in num_cols if col not in cols_to_drop]
cat_cols = [col for col in cat_cols if col not in cols_to_drop]

# --- 2.2. Lựa chọn Biến Số (Numerical Feature Selection) ---
STD_THRESHOLD = 0.01 
CORR_THRESHOLD = 0.9 

# Phương sai Thấp
low_variance_cols = [col for col in num_cols if df[col].std() < STD_THRESHOLD]
cols_to_drop.update(low_variance_cols)
if low_variance_cols:
    print(f"\n🚫 Loại bỏ do Phương sai Thấp (STD < {STD_THRESHOLD}): {low_variance_cols}")

# Tương quan Cao
num_cols_after_var = [col for col in num_cols if col not in cols_to_drop]
corr_matrix = df[num_cols_after_var].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

for column in upper.columns:
    correlated_cols = upper.index[upper[column] > CORR_THRESHOLD].tolist()
    for row in correlated_cols:
        if column not in cols_to_drop:
            cols_to_drop.add(column) 
            print(f"   - Tương quan cao ({upper.loc[row, column]:.2f}): Giữ '{row}', Loại bỏ '{column}'")

# --- 2.3. Lựa chọn Biến Phân loại (Categorical Feature Selection) ---
FREQ_THRESHOLD = 0.95 

# Tần suất Thống trị
cat_to_drop_freq = [col for col in cat_cols if df[col].value_counts(normalize=True).iloc[0] > FREQ_THRESHOLD]
cols_to_drop.update(cat_to_drop_freq)
if cat_to_drop_freq:
    print(f"\n🚫 Loại bỏ cột Cat do Tần suất Thống trị (>95%): {cat_to_drop_freq}")

# Tương quan Cramer's V (đơn giản hóa vì hiếm khi dùng cho lượng lớn cột)
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(r - 1, k - 1))

# (Bỏ qua việc tính Cramer's V lặp vì quá tốn kém trên code mẫu 49 cột)

# ==============================================================================
# 3. KẾT QUẢ FEATURE SELECTION VÀ CHIA TRAIN-TEST SPLIT
# ==============================================================================
final_selected_cols = [col for col in df.columns if col not in cols_to_drop]
final_num_cols = [col for col in num_cols if col not in cols_to_drop]
final_cat_cols = [col for col in cat_cols if col not in cols_to_drop]

# Áp dụng Feature Selection
df_clean = df[final_selected_cols]

# CHIA TRAIN/TEST
df_train, df_test = train_test_split(df_clean, test_size=0.2, random_state=42)

print("-" * 50)
print(f"TỔNG KẾT FS: Đã giảm từ {df.shape[1]} cột xuống còn {len(final_selected_cols)} cột.")
print(f"Kích thước tập Train đã làm sạch: {df_train.shape}")
print(f"Cột Số cuối cùng ({len(final_num_cols)}): {final_num_cols[:5]}...")
print(f"Cột Cat cuối cùng ({len(final_cat_cols)}): {final_cat_cols}")
print("-" * 50)


# ==============================================================================
# 4. TIỀN XỬ LÝ (PREPROCESSING) SAU KHI CHIA (Áp dụng cho Học Không Giám Sát)
# ==============================================================================

# Chúng ta chỉ FIT trên tập TRAIN để tránh Data Leakage
preprocessor = ColumnTransformer(
    transformers=[
        # Scaling cho cột số (Quan trọng cho K-Means, PCA)
        ('num', StandardScaler(), final_num_cols),
        # Encoding cho cột phân loại
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), final_cat_cols)
    ],
    remainder='passthrough' # Giữ lại các cột khác (nếu có)
)

# Xây dựng Pipeline (Dù không có mô hình, vẫn dùng Pipeline để áp dụng Preprocessing)
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# FIT và TRANSFORM trên TRAIN
df_train_processed = pipeline.fit_transform(df_train)

# CHỈ TRANSFORM trên TEST
df_test_processed = pipeline.transform(df_test)

# Chuyển về DataFrame để dễ nhìn hơn
feature_names = final_num_cols + list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(final_cat_cols))
df_train_final = pd.DataFrame(df_train_processed, columns=feature_names)
df_test_final = pd.DataFrame(df_test_processed, columns=feature_names)

print(f"Kích thước tập Train SAU PREPROCESSING: {df_train_final.shape} (Số cột đã tăng do One-Hot Encoding)")
print("Dữ liệu đã sẵn sàng cho các thuật toán Unsupervised Learning (K-Means, PCA, v.v.).")

NameError: name 'df' is not defined

# Supervise

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report 

# ==============================================================================
# 1. NẠP DỮ LIỆU VÀ CHIA TRAIN-TEST SPLIT (Feature Selection được bỏ qua cho ngắn gọn)
# ==============================================================================

# --- TẠO DỮ LIỆU GIẢ LẬP VÀ BIẾN MỤC TIÊU ---
np.random.seed(42)
data_size = 500
df = pd.DataFrame({
    'Num_A': np.random.rand(data_size) * 10,
    'Num_B': np.random.normal(50, 10, data_size),
    'Cat_F': np.random.choice(['Red', 'Green', 'Blue'], data_size, p=[0.7, 0.2, 0.1]),
    'Cat_G': np.random.choice(['A', 'B', 'C', 'D'], data_size, p=[0.4, 0.3, 0.2, 0.1]),
    'Target': np.where((np.random.rand(data_size) + np.random.rand(data_size)) > 1, 1, 0)
})

# Xác định cột
TARGET_COL = 'Target'
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Chia Train/Test (sử dụng stratify)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Phân loại cột sau khi chia
num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

print(f"Kích thước X_train: {X_train.shape}")
print("-" * 50)


# ==============================================================================
# 2. TIỀN XỬ LÝ (PREPROCESSING) THỦ CÔNG (KHÔNG DÙNG PIPELINE)
# ==============================================================================

# Khởi tạo các bộ biến đổi
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# --- A. XỬ LÝ CỘT SỐ (SCALING) ---

print("Bắt đầu xử lý cột số (Scaling)...")

# 1. FIT (chỉ trên X_train)
scaler.fit(X_train[num_cols])

# 2. TRANSFORM (trên X_train và X_test)
X_train_scaled = scaler.transform(X_train[num_cols])
X_test_scaled = scaler.transform(X_test[num_cols])

# Chuyển về DataFrame tạm thời (để dễ dàng ghép nối)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=num_cols, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=num_cols, index=X_test.index)


# --- B. XỬ LÝ CỘT PHÂN LOẠI (ENCODING) ---

print("Bắt đầu xử lý cột phân loại (One-Hot Encoding)...")

# 1. FIT (chỉ trên X_train)
encoder.fit(X_train[cat_cols])

# Lấy tên cột mới sau One-Hot Encoding
encoded_cols = encoder.get_feature_names_out(cat_cols)

# 2. TRANSFORM (trên X_train và X_test)
X_train_encoded = encoder.transform(X_train[cat_cols])
X_test_encoded = encoder.transform(X_test[cat_cols])

# Chuyển về DataFrame tạm thời
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoded_cols, index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoded_cols, index=X_test.index)


# --- C. GHÉP NỐI (CONCATENATE) DỮ LIỆU ĐÃ XỬ LÝ ---

# Loại bỏ các cột gốc đã được xử lý khỏi tập dữ liệu ban đầu
X_train_removed = X_train.drop(columns=num_cols + cat_cols)
X_test_removed = X_test.drop(columns=num_cols + cat_cols)

# Ghép nối các tập dữ liệu đã xử lý
X_train_final = pd.concat([X_train_scaled_df, X_train_encoded_df, X_train_removed], axis=1)
X_test_final = pd.concat([X_test_scaled_df, X_test_encoded_df, X_test_removed], axis=1)

print(f"\nKích thước X_train sau xử lý: {X_train_final.shape}")
print("-" * 50)


# ==============================================================================
# 3. HUẤN LUYỆN VÀ ĐÁNH GIÁ MÔ HÌNH
# ==============================================================================

# Khởi tạo mô hình
model = LogisticRegression(random_state=42, solver='liblinear')

# Huấn luyện mô hình trên tập TRAIN cuối cùng
model.fit(X_train_final, y_train)

# Dự đoán trên tập TEST cuối cùng
y_pred = model.predict(X_test_final)

# Đánh giá
print("--- ĐÁNH GIÁ MÔ HÌNH TRÊN TẬP TEST ---")
print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("-" * 50)